# Radiomics Feature Extraction

Extracting features from PET and CT images.

In [1]:
import os
import nrrd
import utils 
import radiomics

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from feature_extraction import feature_extractor

%matplotlib inline

In [2]:
path_masksdir = './../../../data_source/images/masks_nrrd/'
base_thresh_mask = './../../../data_source/images/masks_resegm_nrrd/'
path_ct_imagedir = './../../../data_source/images/ct_anomaly_filtered_nrrd/'
path_pet_imagedir = './../../../data_source/images/pet_anomaly_filtered_nrrd/'

In [3]:
paths_ct_images = utils.sample_paths(
    path_ct_imagedir, path_masksdir, target_format='nrrd'
)
paths_pet_images = utils.sample_paths(
    path_pet_imagedir, path_masksdir, target_format='nrrd'
)

## Setup

In [ ]:
def z_score_transform(image, n=None):
    # Texture features is shown to be more stable after GL normalizations.
    # Ref: Voxel size and gray level normalization of CT radiomic features 
    #      in lung cancer.
    _image = np.copy(image)
    
    # Z-score transformation.
    image_std = np.std(_image)
    image_zscore = (_image - np.mean(_image)) / image_std
    
    # Remove outliers.
    if n is not None:
        outliers = n * image_std 
        image_zscore = np.clip(image_zscore, -1.0 * outliers, outliers)

    return image_zscore


def bin_widths(path_images, nbins, n=3, z_scoring=False, outlier_removal=None):
    """Average min and max to compare histogram across stacks.
    Fixed bin iwdth (and not fixed bin size) to compare texture
    features across stacks.
    
    """
    img_max, img_min = [], []
    for image_path in path_images:
        image, _ = nrrd.read(image_path['Image'])
        
        if z_scoring:
            image = z_score_transform(image, n=outlier_removal)
        
        img_max.append(np.ceil(np.max(image)))
        img_min.append(np.floor(np.min(image)))
  
    return (np.mean(img_max) - np.mean(img_min)) / nbins

In [ ]:
bin_widths(paths_ct_images, 32), bin_widths(paths_ct_images, 64), bin_widths(paths_ct_images, 128)

In [ ]:
bin_widths(paths_pet_images, 32), bin_widths(paths_pet_images, 64), bin_widths(paths_pet_images, 128)

In [ ]:
print(bin_widths(paths_ct_images, 32, z_scoring=True, outlier_removal=3))
print(bin_widths(paths_ct_images, 64, z_scoring=True, outlier_removal=3))
print(bin_widths(paths_ct_images, 128, z_scoring=True, outlier_removal=3))

## Mask resegmentation

In [6]:
def threshold(image, mask, thresh=None):
    # REF: 
    # * Effects of CT threshold value to make a surface bone model 
    #   on accuracy of shape-based registration in a CT-based navigation 
    #   system for hip surgery.
    
    _mask = np.copy(mask)
    _mask[np.where(image > thresh)] = 0
    
    return _mask

In [7]:
base_thresh_mask = './../../../data_source/images/masks_resegm_nrrd/'
for image_path in paths_ct_images:
    image, _ = nrrd.read(image_path['Image'])
    mask, _ = nrrd.read(image_path['Mask'])

    mask_thresh = threshold(image, mask, thresh=1500)
    mask_thresh_path = os.path.join(base_thresh_mask, image_path['Image'].split('/')[-1])

    nrrd.write(mask_thresh_path, mask_thresh)

In [8]:
# Update file paths to re-segmented masks.
paths_ct_images = utils.sample_paths(
    path_ct_imagedir, base_thresh_mask, target_format='nrrd'
)
paths_pet_images = utils.sample_paths(
    path_pet_imagedir, base_thresh_mask, target_format='nrrd'
)

## Calculate features

In [13]:
ct_param_files = [
    #'./parameter_files/shape/shape_config.yaml',
    #'./parameter_files/firstorder/ct32_firstorder_no_filter_config.yaml',
    #'./parameter_files/firstorder/ct64_firstorder_no_filter_config.yaml',
    #'./parameter_files/firstorder/ct128_firstorder_no_filter_config.yaml'
    './parameter_files/texture/ct32_texture_no_filter_config.yaml',
    './parameter_files/texture/ct64_texture_no_filter_config.yaml',
    './parameter_files/texture/ct128_texture_no_filter_config.yaml',
]
ct_feature_files = [
    #'./../../../data_source/radiomic_features/shape/shape_features.csv',
    #'./../../../data_source/radiomic_features/firstorder/ct32_firstorder_no_filter_config.csv',
    #'./../../../data_source/radiomic_features/firstorder/ct64_firstorder_no_filter_config.csv',
    #'./../../../data_source/radiomic_features/firstorder/ct128_firstorder_no_filter_config.csv'
    './../../../data_source/radiomic_features/texture/ct32_texture_no_filter_config.csv',
    './../../../data_source/radiomic_features/texture/ct64_texture_no_filter_config.csv',
    './../../../data_source/radiomic_features/texture/ct128_texture_no_filter_config.csv',
]
for ct_param_file, ct_feature_file in zip(ct_param_files, ct_feature_files):
    feature_extractor(
        ct_param_file,
        paths_ct_images, 
        verbose=1, 
        path_to_results=ct_feature_file
    )

Initiated feature extraction.


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    6.7s
[Parallel(n_jobs=3)]: Done 195 out of 195 | elapsed:   28.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Initiated feature extraction.


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    6.1s
[Parallel(n_jobs=3)]: Done 195 out of 195 | elapsed:   27.6s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Initiated feature extraction.


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    6.3s
[Parallel(n_jobs=3)]: Done 195 out of 195 | elapsed:   28.7s finished


In [14]:
pet_param_files = [
    #'./parameter_files/firstorder/pet32_firstorder_no_filter_config.yaml',
    #'./parameter_files/firstorder/pet64_firstorder_no_filter_config.yaml',
    #'./parameter_files/firstorder/pet128_firstorder_no_filter_config.yaml'
    './parameter_files/texture/pet32_texture_no_filter_config.yaml',
    './parameter_files/texture/pet64_texture_no_filter_config.yaml',
    './parameter_files/texture/pet128_texture_no_filter_config.yaml',
]
pet_feature_files = [
    #'./../../../data_source/radiomic_features/firstorder/pet32_firstorder_no_filter_config.csv',
    #'./../../../data_source/radiomic_features/firstorder/pet64_firstorder_no_filter_config.csv',
    #'./../../../data_source/radiomic_features/firstorder/pet128_firstorder_no_filter_config.csv'
    './../../../data_source/radiomic_features/texture/pet32_texture_no_filter_config.csv',
    './../../../data_source/radiomic_features/texture/pet64_texture_no_filter_config.csv',
    './../../../data_source/radiomic_features/texture/pet128_texture_no_filter_config.csv',
]
for pet_param_file, pet_feature_file in zip(pet_param_files, pet_feature_files):
    feature_extractor(
        pet_param_file,
        paths_pet_images, 
        verbose=1, 
        path_to_results=pet_feature_file
    )

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Initiated feature extraction.


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    5.5s
[Parallel(n_jobs=3)]: Done 195 out of 195 | elapsed:   25.9s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Initiated feature extraction.


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    5.8s
[Parallel(n_jobs=3)]: Done 195 out of 195 | elapsed:   27.9s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Initiated feature extraction.


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    8.1s
[Parallel(n_jobs=3)]: Done 195 out of 195 | elapsed:   35.6s finished
